In [104]:
#!g1.1
!wget https://raw.githubusercontent.com/shitkov/bert4classification/main/bert_classifier.py
!wget https://raw.githubusercontent.com/shitkov/bert4classification/main/bert_dataset.py
!wget https://raw.githubusercontent.com/shitkov/bert4classification/main/requirements.txt

--2023-04-09 10:10:32--  https://raw.githubusercontent.com/shitkov/bert4classification/main/bert_classifier.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5278 (5.2K) [text/plain]
Saving to: ‘bert_classifier.py’

     0K .....                                                 100% 31.5M=0s

2023-04-09 10:10:32 (31.5 MB/s) - ‘bert_classifier.py’ saved [5278/5278]

--2023-04-09 10:10:32--  https://raw.githubusercontent.com/shitkov/bert4classification/main/bert_dataset.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 891 [text/pl

In [5]:
%pip install -r requirements.txt


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 804.1 MB 1.9 kB/s              
     |████████████████████████████████| 1.2 MB 60.0 MB/s            
     |████████████████████████████████| 2.1 MB 107.2 MB/s            
     |████████████████████████████████| 9.3 MB 76.3 MB/s            
     |████████████████████████████████| 14.9 MB 118.2 MB/s            
     |████████████████████████████████| 7.0 MB 75.8 MB/s            
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following d

In [153]:
#!g1.1
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import torch
import matplotlib.pyplot as plt

In [154]:
#!g1.1
TEST_SIZE = 0.2 # Test size for metric check
TIMEOUT = 9000 # Time in seconds for automl run
TARGET_NAME = 'sentiment' # Target column name

In [155]:
#!g1.1
train = pd.read_csv('/home/jupyter/mnt/s3/mikezz21datahackathon/train.csv')

train_sent = train.loc[:,['sentence', 'sentiment']]
train_sent = train_sent[train_sent.duplicated()==False]
duplicates = train_sent[train_sent.duplicated('sentence', keep = False)==True].sort_values(by = 'sentence')
train_sent = train_sent[train_sent.duplicated('sentence', keep = False)==False]

train_sent['sentiment'] = train_sent['sentiment'].replace({'+': 1, '−': 2, '?': 0})

In [156]:
#!g1.1
train_data, test_data = train_test_split(train_sent, 
                                         test_size=TEST_SIZE, 
                                         stratify=train_sent[TARGET_NAME], 
                                         random_state=10)

print('Data splitted. Parts sizes: train_data = {}, test_data = {}'
              .format(train_data.shape, test_data.shape))

Data splitted. Parts sizes: train_data = (4791, 2), test_data = (1198, 2)


In [157]:
#!g1.1
train, valid = train_test_split(train_data, 
                                         test_size=TEST_SIZE, 
                                         stratify=train_data[TARGET_NAME], 
                                         random_state=10)

print('Data splitted. Parts sizes: train = {}, valid = {}'
              .format(train.shape, valid.shape))

Data splitted. Parts sizes: train = (3832, 2), valid = (959, 2)


In [158]:
#!g1.1
from bert_dataset import CustomDataset
from bert_classifier import BertClassifier

In [171]:
#!g1.1
classifier = BertClassifier(
        model_path='cointegrated/LaBSE-en-ru',
        tokenizer_path='cointegrated/LaBSE-en-ru',
        n_classes=3,
        epochs=5,
        model_save_path='bert.pt'
)

Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not i

In [172]:
#!g1.1
classifier.preparation(
        X_train=list(train['sentence']),
        y_train=list(train['sentiment']),
        X_valid=list(valid['sentence']),
        y_valid=list(valid['sentiment'])
    )

In [173]:
#!g1.1

classifier.train()

Epoch 1/5
Train loss 0.54733802481197 accuracy 0.8776096033402921
Val loss 0.43075253869659114 accuracy 0.9113660062565172
----------
Epoch 2/5
Train loss 0.26783889846434145 accuracy 0.949634655532359
Val loss 0.32785549921754864 accuracy 0.9311783107403545
----------
Epoch 3/5
Train loss 0.14831838606348388 accuracy 0.972599164926931
Val loss 0.42455433977138457 accuracy 0.9249217935349322
----------
Epoch 4/5
Train loss 0.08159848449346226 accuracy 0.98643006263048
Val loss 0.40367167137431653 accuracy 0.9374348279457769
----------
Epoch 5/5
Train loss 0.034665543237426696 accuracy 0.9942588726513569
Val loss 0.4086671526201144 accuracy 0.9374348279457769
----------


In [210]:
#!g1.1
data_participants = pd.read_csv('1sentencenewtest_sentence.csv')
texts = list(data_participants['sentence'])
predictions = [classifier.predict(t) for t in texts]



In [211]:
#!g1.1
import torch.nn as nn
x = nn.functional.softmax(torch.cat(predictions,dim=0), dim=1)


In [212]:
#!g1.1
list_of_prob = x.numpy()


In [213]:
#!g1.1
list_of_prob[0].sum()

0.99999994

In [185]:
#!g1.1
from sklearn.metrics import roc_auc_score
roc_auc_score(labels, list_of_prob, multi_class = 'ovr')

0.9700244309584657

In [199]:
#!g1.1
list_of_prob

array([[5.2715397e-05, 2.3473191e-05, 9.9992383e-01],
       [5.3600375e-05, 9.9987447e-01, 7.1932722e-05],
       [4.9991959e-05, 2.3441646e-05, 9.9992657e-01],
       ...,
       [4.0504269e-04, 9.9950147e-01, 9.3482835e-05],
       [9.5448602e-05, 3.7025133e-05, 9.9986756e-01],
       [5.0740291e-05, 9.9988163e-01, 6.7582754e-05]], dtype=float32)

In [214]:
#!g1.1
data_participants['prob_?'] = list_of_prob[:,0]
data_participants['prob_+'] = list_of_prob[:,1]
data_participants['prob_-'] = list_of_prob[:,2]



In [215]:
#!g1.1
data_participants.to_csv('data_participants_with_probs.csv')

In [191]:
#!g1.1
train.to_csv('train_data.csv')

In [55]:
#!g1.1
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.8553061796560385, recall: 0.8485804896132582, f1score: 0.8516182921411679


In [ ]:
#!g1.1
